In [11]:
def deco(func):
    def inner():
        func()
        print('running inner()')
    return inner

In [12]:
@deco
def target():
    print('running target()')

In [ ]:
target()

In [ ]:
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

class A():

    @register
    def f1(self):
        print('running f1()')

    @register
    def f2(self):
        print('running f2()')

    def f3(self):
        print('running f3()')

print(registry)

In [ ]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

In [ ]:
avg_1 = make_averager()
avg_2 = make_averager()

for i in range(10):
    print(avg_1(i), end=', ')
    print(avg_2(i+100))

In [6]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [ ]:
avg = make_averager()
avg(10)

In [10]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

In [16]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

functools.lru_cache 是非常实用的装饰器，它实现了备忘（memoization）功能

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

In [ ]:
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

In [ ]:
from functools import singledispatch
from collections import abc

@singledispatch
def test(obj):
    return 'test'

@test.register(str)
def _(text):
    return 'test_str'

@test.register(list)
def _(text):
    return 'test_list'


@test.register(dict)
@test.register(int)
def _(text):
    return 'test_a'

In [ ]:
test('a')

In [ ]:
import numbers
# 获取类C的父类
parent_classes = numbers.Integral.__mro__

# 打印父类
for parent in parent_classes:
    print(parent.__name__)

In [ ]:
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func
@register
def f1():
    print('running f1()')
print('running main()')
print('registry ->', registry)
f1()

In [43]:
registry = set()
def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [ ]:
@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')
print(registry)

In [5]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    yield 'JABBERWORKY'
    sys.stdout.write = original_write

In [6]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)
what

pordwonS dna yttiK ,ecilA
YKROWREBBAJ


'JABBERWORKY'

In [24]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b = ', b)
    c = yield a + b
    print('-> Received: c = ', c)

In [25]:
my_coro2 = simple_coro2(14)
next(my_coro2)

-> Started: a = 14


14

In [10]:
my_coro2.send(28)

-> Received: b =  28


42

In [11]:
my_coro2.send(99)

-> Received: c =  99


StopIteration: 

In [29]:
def averager():
    total = 0.0
    count = 0
    averager = None
    while True:
        print('count is ', count)
        term = yield averager
        total += term
        count += 1
        averager = total / count

In [30]:
coro_avg = averager()
next(coro_avg)

count is  0


In [31]:
coro_avg.send(10)

count is  1


10.0

In [32]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [33]:
@coroutine
def averager():
    total = 0.0
    count = 0
    averager = None
    while True:
        print('count is ', count)
        term = yield averager
        total += term
        count += 1
        averager = total / count

In [34]:
coro_avg = averager()

count is  0


In [35]:
coro_avg.send(10)

count is  1


10.0

In [36]:
coro_avg.send(30)

count is  2


20.0

In [37]:
coro_avg.send(5)

count is  3


15.0